In [49]:
#| label: libraries
#| include: false
import pandas as pd
import altair as alt
import numpy as np
from scipy import stats
import json

## Elevator pitch

From my findings, based off of percentage of flights delayed and minutes of delay per flight, the Salt Lake City airport is the best to fly out of, and the San Francisco is the worst. Based off of the same criteria, September is the best month to fly in to avoid delays. At each airport, around 1/3 of flights are delayed due to weather.


In [50]:
#| label: project data
#| code-summary: Read and format project data
# Include and execute your code here
data = "C:/Fall 23/ds250/proj2/flights_missing.json"
df = pd.read_json(data)
df_copy = df.copy()

## GRAND QUESTION 1

Which airport has the worst delays?

Based on my analysis, the San Francisco, CA airport has the highest percentage of delays, and the Chicago, IL airport has the longest delays on average. While finding this data, I realized some of the airport names were missing, so I created a funciton in order to add those names in based off of the airport codes to make the report more comprehensive. I then added the percent of flights delayed and the avg delay hours columns to view these specific critera.


In [51]:
#| label: GQ1
#| code-summary: Read and format data
# Include and execute your code here
airport_name_mapping = {
    "IAD": "Washington, DC: Washington Dulles International",
    "SLC": "Salt Lake City, UT: Salt Lake City International",
    "SAN": "San Diego, CA: San Diego International",
    "ORD": "Chicago, IL: Chicago O'Hare International",
    "DEN": "Denver, CO: Denver International",
    "ATL": "Atlanta, GA: Hartsfield-Jackson Atlanta International",
    "SFO": "San Francisco, CA: San Francisco International",
}


def replace_empty_strings(row):
    if row["airport_name"] == "":
        return airport_name_mapping.get(row["airport_code"], "")
    else:
        return row["airport_name"]


df["airport_name"] = df.apply(replace_empty_strings, axis=1)


filtered_df = df.filter(
    [
        "airport_name",
        "num_of_flights_total",
        "num_of_delays_total",
        "minutes_delayed_total",
    ]
)

aggregated_df = (
    filtered_df.groupby("airport_name")
    .agg(
        {
            "num_of_flights_total": "sum",
            "num_of_delays_total": "sum",
            "minutes_delayed_total": "sum",
        }
    )
    .reset_index()
)
aggregated_df["percent_of_flights_delayed"] = (
    aggregated_df["num_of_delays_total"] / aggregated_df["num_of_flights_total"]
) * 100
aggregated_df["percent_of_flights_delayed"] = aggregated_df[
    "percent_of_flights_delayed"
].round(2)


aggregated_df["avg_delay_hours"] = (
    aggregated_df["minutes_delayed_total"] / aggregated_df["num_of_delays_total"]
) / 60
aggregated_df["avg_delay_hours"] = aggregated_df["avg_delay_hours"].round(2)

In [52]:
#| label: GQ1 chart
#| code-summary: plot example
#| fig-cap: GQ1 table
#| fig-align: center
# Include and execute your code here
aggregated_df.sort_values("percent_of_flights_delayed")

,airport_name,num_of_flights_total,num_of_delays_total,minutes_delayed_total,percent_of_flights_delayed,avg_delay_hours
3,"Salt Lake City, UT: Salt Lake City International",1403384,205160,10123371,14.62,0.82
2,"Denver, CO: Denver International",2513974,468519,25173381,18.64,0.90
4,"San Diego, CA: San Diego International",917862,175132,8276248,19.08,0.79
6,"Washington, DC: Washington Dulles International",851571,168467,10283478,19.78,1.02
0,"Atlanta, GA: Hartsfield-Jackson Atlanta Intern...",4430047,902443,53983926,20.37,1.00
1,"Chicago, IL: Chicago O'Hare International",3597588,830825,56356129,23.09,1.13
5,"San Francisco, CA: San Francisco International",1630945,425604,26550493,26.10,1.04


## GRAND QUESTION 2

What is the best month to fly if you want to avoid delays of any length?

The best month to fly in order to avoid any length of delay would be September. While months like November and October may have slightly shorter delays on average, September has a lower percentage of flights delayed overall.


In [53]:
#| label: GQ2
#| code-summary: Read and format data
# Include and execute your code here
filtered_month_df = df.filter(
    [
        "airport_name",
        "month",
        "num_of_flights_total",
        "num_of_delays_total",
        "minutes_delayed_total",
    ]
)
filtered_month_df


aggregated_month_df = (
    filtered_month_df.groupby("month")
    .agg(
        {
            "num_of_flights_total": "sum",
            "num_of_delays_total": "sum",
            "minutes_delayed_total": "sum",
        }
    )
    .reset_index()
)

aggregated_month_df = aggregated_month_df.query('month != "n/a"')


aggregated_month_df["percent_of_flights_delayed"] = (
    aggregated_month_df["num_of_delays_total"]
    / aggregated_month_df["num_of_flights_total"]
) * 100
aggregated_month_df["percent_of_flights_delayed"] = aggregated_month_df[
    "percent_of_flights_delayed"
].round(2)

aggregated_month_df["avg_delay_hours"] = (
    aggregated_month_df["minutes_delayed_total"]
    / aggregated_month_df["num_of_delays_total"]
) / 60
aggregated_month_df["avg_delay_hours"] = aggregated_month_df["avg_delay_hours"].round(2)

In [54]:
#| label: GQ2 chart
#| code-summary: plot example
#| fig-cap: GQ2 Table
#| fig-align: center
# Include and execute your code here
aggregated_month_df.sort_values("percent_of_flights_delayed")

,month,num_of_flights_total,num_of_delays_total,minutes_delayed_total,percent_of_flights_delayed,avg_delay_hours
11,September,1227208,201905,11495811,16.45,0.95
9,November,1185434,197768,11112089,16.68,0.94
10,October,1301612,235166,13109792,18.07,0.93
0,April,1259723,231408,13667654,18.37,0.98
8,May,1227795,233494,13637705,19.02,0.97
7,March,1213370,250142,14942262,20.62,1.00
1,August,1335158,279699,16906565,20.95,1.01
4,January,1193018,265001,16152667,22.21,1.02
3,Febuary,1115814,248033,14753955,22.23,0.99
5,July,1371741,319960,20465456,23.33,1.07


## GRAND QUESTION 3

Your job is to create a new column that calculates the total number of flights delayed by weather (both severe and mild).

I created these columns by adding the weather category, with 30% of the late arriving and 40% of NAS from April to August and 65% of NAS from the remaining months. This resulted in the num of delays weather total column.


In [55]:
#| label: GQ3
#| code-summary: Read and format data
# Include and execute your code here
df_without_missing = df.query("num_of_delays_late_aircraft != -999")

mean_late_aircraft = df_without_missing["num_of_delays_late_aircraft"].mean()
mean_late_aircraft = mean_late_aircraft.round(2)

df_total_weather = df.filter(
    [
        "airport_code",
        "month",
        "num_of_delays_total",
        "num_of_delays_late_aircraft",
        "num_of_delays_nas",
        "num_of_delays_weather",
    ]
)
df_total_weather["num_of_delays_late_aircraft"] = df_total_weather[
    "num_of_delays_late_aircraft"
].replace(-999, mean_late_aircraft)

df_total_weather["num_of_delays_late_new"] = (
    30 / 100 * df_total_weather["num_of_delays_late_aircraft"].round(0)
)


def calculate_num_of_delays_nas_new(row):
    if row["month"] in ["April", "May", "June", "July", "August"]:
        return 0.4 * row["num_of_delays_nas"]
    else:
        return 0.65 * row["num_of_delays_nas"]


df_total_weather["num_of_delays_nas_new"] = df_total_weather.apply(
    calculate_num_of_delays_nas_new, axis=1
)
df_total_weather["num_of_weather_delays_total"] = (
    df_total_weather["num_of_delays_late_new"]
    + df_total_weather["num_of_delays_nas_new"]
    + df_total_weather["num_of_delays_weather"]
)
df_total_weather["num_of_weather_delays_total"] = df_total_weather[
    "num_of_weather_delays_total"
].astype(int)

In [56]:
#| label: GQ3 chart
#| code-summary: plot example
#| fig-cap: GQ3 Table
#| fig-align: center
# Include and execute your code here
df_total_weather.head(5)

,airport_code,month,num_of_delays_late_aircraft,num_of_delays_nas,num_of_delays_weather,num_of_delays_total,num_of_delays_late_new,num_of_delays_nas_new,num_of_weather_delays_total
0,ATL,January,1109.1,4598,448,8355,332.7,2988.70,3769
1,DEN,January,928.0,935,233,3153,278.4,607.75,1119
2,IAD,January,1058.0,895,61,2430,317.4,581.75,960
3,ORD,January,2255.0,5415,306,9178,676.5,3519.75,4502
4,SAN,January,680.0,638,56,1952,204.0,414.70,674


## GRAND QUESTION 4

Using the new weather variable calculated above, create a barplot showing the proportion of all flights that are delayed by weather at each airport. Discuss what you learn from this graph.

I created this graph by layering the total weather delay data over to total delay data. I found that around 1/3 of flights at all of these airports are delayed due to weather.


In [57]:
#| label: GQ4
#| code-summary: Read and format data
# Include and execute your code here
filtered_total_weather = df_total_weather.filter(
    ["airport_code", "num_of_delays_total", "num_of_weather_delays_total"]
)

filtered_total_weather

agg_weather_df = (
    df_total_weather.groupby("airport_code")
    .agg({"num_of_delays_total": "sum", "num_of_weather_delays_total": "sum"})
    .reset_index()
)

agg_weather_df


weather_chart = (
    alt.Chart(agg_weather_df)
    .mark_bar()
    .encode(
        x=alt.X("airport_code:N"),
        y=alt.Y(alt.repeat("layer"))
        .aggregate("mean")
        .title("Mean of US and Worldwide Gross"),
        color=alt.ColorDatum(alt.repeat("layer")),
    )
    .repeat(layer=["num_of_delays_total", "num_of_weather_delays_total"])
)

In [58]:
#| label: GQ4 chart
#| code-summary: plot example
#| fig-cap: GQ 4 Chart
#| fig-align: center
# Include and execute your code here
weather_chart

C:\Users\joshu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\altair\utils\core.py:410: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.RepeatChart(...)

## GRAND QUESTION 5

Fix all of the varied missing data types in the data to be consistent (all missing values should be displayed as “NaN”).


In [59]:
#| label: GQ5
#| code-summary: Read and format data
# Include and execute your code here
df_copy.replace("", "NaN", inplace=True)

df_copy.to_json("modified_file.json", orient="records", lines=True)


row_to_print = df_copy.iloc[25].to_dict()

json_data = json.dumps(row_to_print)

In [60]:
#| label: GQ5 chart
#| code-summary: plot example
#| fig-cap: Updated JSON line
#| fig-align: center
# Include and execute your code here
print(json_data)

{"airport_code": "SAN", "airport_name": "NaN", "month": "April", "year": 2005.0, "num_of_flights_total": 7091, "num_of_delays_carrier": "364", "num_of_delays_late_aircraft": 369, "num_of_delays_nas": 343, "num_of_delays_security": 2, "num_of_delays_weather": 15, "num_of_delays_total": 1095, "minutes_delayed_carrier": 15602.0, "minutes_delayed_late_aircraft": 15994, "minutes_delayed_nas": 10015.0, "minutes_delayed_security": 59, "minutes_delayed_weather": 792, "minutes_delayed_total": 42462}


## APPENDIX A (Additional Python Code)

```python
```